In [1]:
import os

In [2]:
pwd

'c:\\Users\\shrin\\Documents\\Python_Scripts\\FetchSearch\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'c:\\Users\\shrin\\Documents\\Python_Scripts\\FetchSearch'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path
    synthetic_df:Path

In [6]:
from fetchSearch.constants import *
from fetchSearch.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
   
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name,
            synthetic_df = config.synthetic_df          
        )

        return model_evaluation_config

In [14]:
import os
from fetchSearch.logging import logger
from sentence_transformers import SentenceTransformer, models, InputExample, losses, evaluation, util, CrossEncoder
from datasets import load_dataset, load_from_disk
import torch
from torch.utils.data import DataLoader
import tqdm
import numpy as np
import pandas as pd

In [17]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
      
    def calculate_cosine_scores(self, evaluator):
        model = CrossEncoder(self.config.tokenizer_path)
        cross_scores = model.predict([[ doc, query] for query, doc in zip(evaluator.sentences1, evaluator.sentences2)])
        # normalize to 0 to 1
        rescaled_array = (cross_scores-np.min(cross_scores))/(np.max(cross_scores)-np.min(cross_scores))
        rescaled_array = np.round(rescaled_array,2)

        return rescaled_array

    def calculate_accuracy(self, predicted_scores, actual_scores):
        errors = []
        accuracy = 0
        for i, score in enumerate(actual_scores):
            error = abs(score - predicted_scores[i] * 100)
            errors.append(error)
        if(len(errors)) == 0:
            logger.info("No sentences to predict : ")
        else:
            accuracy = 100 - sum(errors)/len(errors)
            logger.info(f"Val accuracy with cross-encoders : {accuracy}" )
        return accuracy


    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        sbert_model = SentenceTransformer(self.config.model_path).to(device)
        # Here the biencoder will calculate embdd and send to cross.
        
        #loading data 
        
        val_dataloader = torch.load(os.path.join(self.config.data_path,"val.pth"))
        test_dataloader = torch.load(os.path.join(self.config.data_path,"test.pth"))

        val_evaluator = torch.load(os.path.join(self.config.data_path,"val_eval.pth"))
        test_evaluator = torch.load(os.path.join(self.config.data_path,"test_eval.pth"))

        predicted_score = self.calculate_cosine_scores(val_evaluator)
        accuray = self.calculate_accuracy(predicted_score, val_evaluator.scores)

        val_df = pd.DataFrame({
            "Offer_embd"    :   val_evaluator.sentences1,
            "Search_query"  :   val_evaluator.sentences1,
            "Actual_scores" :   val_evaluator.scores,
            "Actual_scores" :   predicted_score.tolist()
        })
        val_df.to_csv(self.config.metric_file_name, index=False)


In [18]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2023-10-10 03:34:30,586: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-10 03:34:30,589: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-10 03:34:30,592: INFO: common: created directory at: artifacts]
[2023-10-10 03:34:30,593: INFO: common: created directory at: artifacts/model_evaluation]
[2023-10-10 03:34:30,594: INFO: SentenceTransformer: Load pretrained SentenceTransformer: cross-encoder/ms-marco-MiniLM-L-6-v2]
[2023-10-10 03:34:31,005: WARNING: SentenceTransformer: No sentence-transformers model found with name C:\Users\shrin/.cache\torch\sentence_transformers\cross-encoder_ms-marco-MiniLM-L-6-v2. Creating a new one with MEAN pooling.]
[2023-10-10 03:34:31,321: INFO: SentenceTransformer: Use pytorch device: cpu]
[2023-10-10 03:34:31,809: INFO: CrossEncoder: Use pytorch device: cpu]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[2023-10-10 03:34:31,886: INFO: 2933710435: Val accuracy with cross-encoders : 98.7500003632158]
